In [1]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
lem = WordNetLemmatizer()
RANDOM_STATE = 50
UNK_ID = 1
PAD_ID = 0
MAX_LEN = 20

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\samar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\samar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [1]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [2]:
def clean_string(s):    
    s =  re.sub(r'(?<=[^\s0-9])(?=[.,;?])', r' ', s)
    s = re.sub(r'\((\d+)\)', r'', s)
    s = re.sub(r'\s\s', ' ', s)
    s = re.sub(r"[^A-Za-z0-9(),!?\'`]", " ", s)
    s = re.sub(r"\'s", " \'s", s)
    s = re.sub(r"\'ve", " \'ve", s)
    s = re.sub(r"n\'t", " n\'t", s)
    s = re.sub(r"\'re", " \'re", s)
    s = re.sub(r"\'d", " \'d", s)
    s = re.sub(r"\'ll", " \'ll", s)
    s = re.sub(r",", " , ", s)
    s = re.sub(r"!", " ! ", s)
    s = re.sub(r"\"", " \" ", s)
    s = re.sub(r"\(", " ( ", s)
    s = re.sub(r"\)", " ) ", s)
    s = re.sub(r"\?", " ? ", s)
    s = re.sub(r"\s{2,}", " ", s)
    s = re.sub(r"\.", " . ", s)
    s = re.sub(r"., ", " , ", s)
    s = re.sub(r"\\n", " ", s)
    return s.strip().lower()

def create_train_valid(features,labels,train_fraction = 0.9,max_valid=500):

    features,labels = shuffle(features,labels,random_state = RANDOM_STATE)

    train_end = max(int(train_fraction*len(labels)),len(labels)-max_valid)

    train_features = np.asarray(features[:train_end])
    valid_features = np.asarray(features[train_end:])

    train_labels = np.asarray(labels[:train_end])
    valid_labels = np.asarray(labels[train_end:])
    
    return train_features,valid_features,train_labels,valid_labels

In [3]:
data = pd.read_csv('../datasets/sarcasm_data.csv').dropna(axis=0,how='any')

features = data['text'].to_list()
labels = data['sarcasm'].to_list()

X_train, X_valid, y_train, y_valid = create_train_valid(features,labels)
training_dict = {'X_train': X_train, 'X_valid': X_valid,'y_train': y_train, 'y_valid': y_valid}



In [74]:
info = features
ans = sentiment_pipeline(info)

In [4]:
df = pd.DataFrame(columns = ['text','fear',
'anger',
'anticip',
'trust',
'surprise',
'positive',
'negative',
'sadness',
'disgust',
'joy', 'label'])

In [5]:
from nrclex import NRCLex
text = 'I love India'
# text = info[1]
for text,label in zip(features,labels):
    emotion = NRCLex(text)
    ans = emotion.top_emotions
    ans = dict(ans)
    ans.update({'text':text})
    ans.update({'label':label})
    ab = pd.DataFrame(ans.values(), index = ans.keys()).T
    df = df.append(ab)



C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is de

C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is de

C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is de

C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is de

C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is de

C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ab)
C:\Users\samar\AppData\Local\Temp\ipykernel_24296\1750851296.py:11: FutureWarning: The frame.append method is de

In [19]:
df1 = df.fillna(0)
df1


,text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,label,anticipation
0,It's just a privilege to watch your mind at work.,0.5,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,1,0.5
0,I don't think I'll be able to stop thinking ab...,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,1,0.0
0,"Since it's not bee season, you can have my epi...",0.5,0.5,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,0,0.0
0,"Lois Lane is falling, accelerating at an initi...",0.0,0.0,0.0,0.00,0.0,0.333333,0.00,0.0,0.00,0.0,0,0.0
0,I'm just inferring this is a couch because the...,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,1.0,0.00,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"Hes not right for the part, and if I suggest h...",0.0,0.0,0.0,0.25,0.0,0.250000,0.25,0.0,0.25,0.0,1,0.0
0,"Oh yeah he has a caretaker his older brother, ...",0.0,0.0,0.0,0.40,0.0,0.400000,0.00,0.0,0.00,0.0,0,0.0
0,Is it me or the greetings gone downhill around...,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,1,0.0
0,"You are right, by saying nice, I am virtually ...",0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.0,1,0.0


In [7]:
# ab = pd.DataFrame(ans.values(), index = ans.keys()).T
# df.append(ab)

In [13]:
df.to_csv("data.csv")

In [20]:
X = df1.drop(['label'],axis = 1)
y = df1['label']

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [22]:
X_train

,text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,"Uh, okay. The fifth dentist caved and not they...",0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
0,"Because you don't have a girlfriend? Well, goo...",0.0,0.0,0.0,0.00,0.0,0.333333,0.0,0.0,0.0,0.00,0.0
0,Did I wake you?,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
0,Joey's got a really bad hernia but its nothing...,0.2,0.0,0.0,0.00,0.0,0.000000,0.0,0.2,0.0,0.00,0.0
0,The big deal is that nobody touches food on my...,0.0,0.0,0.0,0.25,0.0,0.250000,0.0,0.0,0.0,0.25,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,When I have enough to destroy all the human to...,0.0,0.0,0.0,0.00,0.0,0.500000,0.0,0.0,0.0,0.50,0.0
0,Is it possible that she was going for your che...,0.0,0.0,0.0,0.00,0.5,0.000000,0.0,0.0,0.0,0.00,0.5
0,"Yeah, good for you, Leonard. That's a lovely l...",0.0,0.0,0.0,0.00,0.0,0.307692,0.0,0.0,0.0,0.00,0.0
0,Just tell me how.,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0


In [14]:
glove_loc = '../glove.6B.100d.txt'
# glove = np.loadtxt(glove_loc,dtype='str',comments=None)
import io
with io.open(glove_loc,encoding = 'utf8') as glove:
    for line in glove:
        values = line.split()
        words = values[0]
        vectors = np.asarray(values[1:],dtype = 'float32')
# vectors = glove[:,1:].astype('float')
# words = glove[:,0]

del glove

In [23]:
vocab = dict()

for sentences in features:
    sentences = clean_string(sentences)
    sentences = nltk.word_tokenize(sentences)
    for word in sentences:
        vocab[word] = 1

In [24]:
word_lookup = {word:vector for word,vector in zip(words,vectors)}

word_index = dict()
ind = 2
not_found = 0

embeds = dict()

for i,word in enumerate(vocab.keys()):
    vector = word_lookup.get(word,None)

    if vector is not None:
        word_index[word] = ind
        embeds[ind] = np.copy(vector)
        ind+=1
    else:
        not_found +=1
        word_index[word] = UNK_ID

print(f'{not_found} words not found.')

1988 words not found.


In [25]:
def word_to_index(utt1):
    utt2 = [word_index.get(word,UNK_ID) for word in nltk.word_tokenize(clean_string(utt1))]
    utt3 = utt2[:MAX_LEN]
    utt4 = utt3 + [PAD_ID]*(MAX_LEN - len(utt3))
    utt5 = np.mean([embeds[i] for i in utt4 if i>1],axis=0)
    if type(utt5) == np.float64:
        utt5 = np.random.rand(100)
    if type(utt5) == np.float64:
        print('here')
    return utt5


In [26]:
X_train['text'] = [word_to_index(word) for word in X_train['text']]
X_test['text']= [word_to_index(word) for word in X_test['text']]

D:\pythonProject\Trial\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\pythonProject\Trial\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [214]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler

In [215]:
def svm_train(features,labels):
    clf = make_pipeline(
        StandardScaler(),
        svm.SVC(C=10.0, gamma="scale", kernel="rbf")
    )
    return clf.fit(features, labels)

In [216]:
def svm_test(clf,features,labels):
    pred = clf.predict(features)
    true = labels

    result_string = classification_report(true, pred, digits=3)
    print(confusion_matrix(true, pred))
    print(result_string)

In [46]:
print(len(X_train['text'].iloc[3]))

TypeError: object of type 'numpy.float32' has no len()

In [224]:
clf = svm.SVC(C=10.0, gamma="scale", kernel="rbf")
clf.fit(X_train,y_train)

ValueError: setting an array element with a sequence.

In [217]:
clf = svm_train(X_train,y_train)

svm_test(clf,X_test,y_test)

ValueError: setting an array element with a sequence.